In [106]:
import sys
import textract, PyPDF2, glob
import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('maxent_ne_chunker')
# nltk.download('words')
# nltk.download('stopwords')
import re
import os
import subprocess 
from pdfminer.high_level import extract_text

In [129]:
files = glob.glob(r"C:\Users\JiatingChen\Documents\resume_samplers\*")

In [113]:
files[0]

'C:\\Users\\JiatingChen\\Documents\\resume_samplers\\721091408_phonescreening.pdf'

In [114]:
txt = extract_text(files[0])

## NAME

In [81]:
def extract_names(txt): #nlp doesn't do a good job here
    person_names = []

    for sent in nltk.sent_tokenize(txt):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if hasattr(chunk, 'label') and chunk.label() == 'PERSON':
                person_names.append(
                    ' '.join(chunk_leave[0] for chunk_leave in chunk.leaves())
                )

    return person_names

In [97]:
extract_names(txt)[0]

'Karthik'

## PHONE NUMBER

In [84]:
PHONE_REG = re.compile(r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]')

In [89]:
def extract_phone_number(resume_text):
    phone = re.findall(PHONE_REG, resume_text)

    if phone:
        number = ''.join(phone[0])

        if resume_text.find(number) >= 0 and len(number) < 16:
            return number
    return None

In [90]:
extract_phone_number(txt)

'(410)-292-1151'

## EMAIL

In [91]:
EMAIL_REG = re.compile(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+')

In [92]:
def extract_emails(resume_text):
    return re.findall(EMAIL_REG, resume_text)

In [93]:
 extract_emails(txt)[0]

'karthikr2194@gmail.com'

## SCHOOLS AND EDUCATION

In [94]:
RESERVED_WORDS = [
    'school',
    'college',
    'univers',
    'academy',
    'faculty',
    'institute',
    'faculdades',
    'Schola',
    'schule',
    'lise',
    'lyceum',
    'lycee',
    'polytechnic',
    'kolej',
    'ünivers',
    'okul',
]

In [152]:
def extract_education(input_text):
    organizations = []

    # first get all the organization names using nltk
    for sent in nltk.sent_tokenize(input_text):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if hasattr(chunk, 'label') and chunk.label() == 'ORGANIZATION':
                organizations.append(' '.join(c[0] for c in chunk.leaves()))

    # we search for each bigram and trigram for reserved words
    # (college, university etc...)
    education = set()
    for org in organizations:
        for word in RESERVED_WORDS:
            if org.lower().find(word) >= 0:
                education.add(org)

    return education


In [96]:
extract_education(txt)

{'Bangalore Institute', 'EDUCATION University'}

In [140]:
output_DataFrame = pd.DataFrame()

In [158]:
def parse_resume(files):
    output_DataFrame = pd.DataFrame()
    
    for file in files: 
        txt = extract_text(file)    
        dict1 = {}
        dict1['name'] = extract_names(txt)[0]
        dict1['phone_num'] = extract_phone_number(txt)
        dict1['email'] =  extract_emails(txt)[0]
        dict1['education'] =  extract_education(txt)
        print(dict1)
        output_DataFrame = output_DataFrame.append(dict1, ignore_index=True)
    return output_DataFrame

In [159]:
for file in files:
    print(file)

C:\Users\JiatingChen\Documents\resume_samplers\721091408_phonescreening.pdf
C:\Users\JiatingChen\Documents\resume_samplers\821101104_phonescreening.pdf
C:\Users\JiatingChen\Documents\resume_samplers\821101136_phonescreening.pdf
C:\Users\JiatingChen\Documents\resume_samplers\821110440_Mythri.pdf
C:\Users\JiatingChen\Documents\resume_samplers\821110445_Qizhe.pdf
C:\Users\JiatingChen\Documents\resume_samplers\821110447_rachan.pdf
C:\Users\JiatingChen\Documents\resume_samplers\821110448_RAGHUVEERA.pdf
C:\Users\JiatingChen\Documents\resume_samplers\821110464_Saida.pdf
C:\Users\JiatingChen\Documents\resume_samplers\821110509_Akash.pdf
C:\Users\JiatingChen\Documents\resume_samplers\821110510_AKSHARA.pdf
C:\Users\JiatingChen\Documents\resume_samplers\821110511_Alexandra.pdf
C:\Users\JiatingChen\Documents\resume_samplers\821110527_.pdf
C:\Users\JiatingChen\Documents\resume_samplers\821110528_Wanting.pdf
C:\Users\JiatingChen\Documents\resume_samplers\821111539_manaswini.pdf
C:\Users\JiatingChen\

In [163]:
output = parse_resume(files)

{'name': 'Karthik', 'phone_num': '(410)-292-1151', 'email': 'karthikr2194@gmail.com', 'education': {'EDUCATION University', 'Bangalore Institute'}}
{'name': 'Robert', 'phone_num': None, 'email': 'hpundir@umd.edu', 'education': {'University'}}
{'name': 'Data Analysis', 'phone_num': '469-370-9437', 'email': 'tirth2410@gmail.com', 'education': {'University', 'Harvard Summer School', 'The University'}}
{'name': 'Mythri', 'phone_num': '(281) 725-7080', 'email': 'mythripartha8@gmail.com', 'education': {'EDUCATION University'}}
{'name': 'Qizhe', 'phone_num': None, 'email': 'ziyaotingyu@gmail.com', 'education': {'EDUCATION University', 'University', 'EXPERIENCE University'}}
{'name': 'Stamford', 'phone_num': '(475) 685 0166', 'email': 'rachan_vamsi.bhooshi@uconn.edu', 'education': {'University', 'UNIVERSITY OF'}}
{'name': 'Github', 'phone_num': '240-713-8296', 'email': 'rmadireddy1@student.gsu.edu', 'education': set()}
{'name': 'Data Science', 'phone_num': '443-833-6344', 'email': 'saidam1@umb

In [165]:
output.to_csv('output_sample.csv')

In [128]:
extract_text(r'C:\\Users\\JiatingChen\\Documents\\resume_samplers\\C01-21120201_Jiayue_Fei.pdf')

'\x0c'